In [1]:
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score, mean_absolute_error
from scipy.optimize import minimize



In [2]:

# ID du Google Sheet et GID de l'onglet
sheet_id = "1PdnadzZswADLqGBrI53a_8IsegivCMSz1pCZOh384-Y"
gid = "1517184233"

# Construction du lien CSV public de Google Sheets
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={gid}"

# Lecture du CSV directement dans un DataFrame
df = pd.read_csv(url)

# Si ton tableau commence à la colonne D (donc avec des colonnes vides avant),
# on peut supprimer les colonnes vides automatiquement :
df = df.dropna(axis=1, how="all")
df = df.replace("-", np.nan)
df = df.dropna(subset=['Team Gap Early', 'Est. Win'])

In [4]:
# ==========================
# PARAMÈTRES GÉNÉRAUX
# ==========================
features = ["Team Gap Early", "R", "Perf relative", "Perf intrinsèque"]
target_col = "Team Gap Early"
max_lag = 7

# ==========================
#  MISE EN FORME DU DATASET
# ==========================
for c in df.columns:
    df[c] = df[c].astype(str).str.replace(",", ".")
    df[c] = pd.to_numeric(df[c], errors="coerce")

# ---- Création des colonnes de lag ----
for feat in features:
    for lag in range(1, max_lag + 1):
        df[f"{feat}_lag{lag}"] = df[feat].shift(lag)

df = df.dropna(subset=[target_col])

# ==========================
# OPTIMISATION DU TAUX DE DÉCROISSANCE
# En temps noomal, on souhaiterai que notre modele apprene le taux de decroissance de lui meme mais on a pas assez de donnee
# On va donc forcer des poids decroissant pour coller a notre intuition du score
# ==========================
def objective(lmbda):
    decay = np.exp(-lmbda * np.arange(1, max_lag + 1))
    decay /= decay.sum()
    
    df_temp = df.copy()
    for feat in features:
        lag_cols = [f"{feat}_lag{i}" for i in range(1, max_lag + 1)]
        df_temp[f"{feat}_weighted"] = df_temp[lag_cols].fillna(0).dot(decay)
    
    X = df_temp[[f"{feat}_weighted" for feat in features]].values
    y = df_temp[target_col].values
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    model = Ridge(alpha=1.0)
    model.fit(X_scaled, y)
    
    return -r2_score(y, model.predict(X))

res = minimize(objective, x0=[0.3], bounds=[(0.05, 1.0)])
best_lambda = res.x[0]
print(f" Lambda optimal trouvé : {best_lambda:.4f}")

# ==========================
# RE-ENTRAÎNEMENT FINAL
# On entraine le modele avec le taux de decroissance des poids pour la lag qui augment
# ==========================
decay = np.exp(-best_lambda * np.arange(1, max_lag + 1))
decay /= decay.sum()

for feat in features:
    lag_cols = [f"{feat}_lag{i}" for i in range(1, max_lag + 1)]
    df[f"{feat}_weighted"] = df[lag_cols].fillna(0).dot(decay)

X = df[[f"{feat}_weighted" for feat in features]].values
y = df[target_col].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

model = Ridge(alpha=1.0)
model.fit(X_scaled, y)

coefs = model.coef_
intercept = model.intercept_

coef_df = pd.DataFrame({
    "Feature": features,
    "Poids": coefs
}).round(4)

print("\n Poids globaux par feature :")
print(coef_df)
print(f"\nIntercept : {intercept:.4f}")

y_pred = model.predict(X_scaled)
print(f"\nR² (train) : {r2_score(y, y_pred):.3f}")
print(f"MAE (train) : {mean_absolute_error(y, y_pred):.3f}")

print(f"\nDécroissance exponentielle utilisée (λ = {best_lambda:.3f}) :")
print(np.round(decay, 4))

# ==========================
# CALCUL DES POIDS EFFECTIFS PAR LAG
# ==========================
# produit w_feature × decay_lag pour chaque feature et lag
weights_matrix = np.outer(coefs, decay)
weights_df = pd.DataFrame(
    weights_matrix,
    index=features,
    columns=[f"lag{i}" for i in range(1, max_lag + 1)]
).round(5)

print("\n------------ Poids effectifs (w_feature × λ_lag) :")
print(weights_df)



 Lambda optimal trouvé : 1.0000

 Poids globaux par feature :
            Feature   Poids
0    Team Gap Early -1.0145
1                 R  0.3953
2     Perf relative -0.2217
3  Perf intrinsèque  0.4078

Intercept : -0.1818

R² (train) : 0.304
MAE (train) : 1.236

Décroissance exponentielle utilisée (λ = 1.000) :
[0.6327 0.2328 0.0856 0.0315 0.0116 0.0043 0.0016]

------------ Poids effectifs (w_feature × λ_lag) :
                     lag1     lag2     lag3     lag4     lag5     lag6  \
Team Gap Early   -0.64184 -0.23612 -0.08686 -0.03196 -0.01176 -0.00432   
R                 0.25008  0.09200  0.03384  0.01245  0.00458  0.00169   
Perf relative    -0.14030 -0.05161 -0.01899 -0.00699 -0.00257 -0.00095   
Perf intrinsèque  0.25803  0.09492  0.03492  0.01285  0.00473  0.00174   

                     lag7  
Team Gap Early   -0.00159  
R                 0.00062  
Perf relative    -0.00035  
Perf intrinsèque  0.00064  


In [16]:
def generate_excel_formula(row, decay, coefs, intercept, start_row=2):

    features_letters = ["G", "H", "I", "J"]  # ordre : Team Gap Early, R, Perf relative, Perf intrinsèque
    
    formula = f"={intercept:.4f}".replace(".", ",")
    
    for feat_idx, col_letter in enumerate(features_letters):
        coef = coefs[feat_idx]
        formula += f"+({coef:.4f})*SOMMEPROD(("
        
        # Les 7 lags : lag1 = row-1, lag2 = row-2, ..., lag7 = row-7
        for lag_idx, d in enumerate(decay):
            lag_row = row - (lag_idx + 1)
            formula += f"{d:.4f}*{col_letter}{lag_row}+"
        
        formula = formula.rstrip("+") + "))"
    
    formula = formula.replace(".", ",")
    return formula


row_number = 9
excel_formula_row9 = generate_excel_formula(row_number, decay, coefs, intercept)
print(excel_formula_row9)

=-0,1818+(-1,0145)*SOMMEPROD((0,6327*G8+0,2328*G7+0,0856*G6+0,0315*G5+0,0116*G4+0,0043*G3+0,0016*G2))+(0,3953)*SOMMEPROD((0,6327*H8+0,2328*H7+0,0856*H6+0,0315*H5+0,0116*H4+0,0043*H3+0,0016*H2))+(-0,2217)*SOMMEPROD((0,6327*I8+0,2328*I7+0,0856*I6+0,0315*I5+0,0116*I4+0,0043*I3+0,0016*I2))+(0,4078)*SOMMEPROD((0,6327*J8+0,2328*J7+0,0856*J6+0,0315*J5+0,0116*J4+0,0043*J3+0,0016*J2))
